In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd

from window.postprocess.match_truths import match_yolo_to_truths
from window.utils.truths import windows_truth

In [ ]:
experiment = 'bin'

filename_inria_metrics = 'inria_' + experiment + '_25_05_metrics.csv'
filename_gfrc_metrics = 'gfrc_' + experiment + '_25_05_metrics.csv'
filename_vedai_metrics = 'vedai_' + experiment + '_25_05_metrics.csv'

filename_inria_detections = 'inria_' + experiment + '_25_05_detections.csv'
filename_gfrc_detections = 'gfrc_' + experiment + '_25_05_detections.csv'
filename_vedai_detections = 'vedai_' + experiment + '_25_05_detections.csv'

filename_inria_windows = 'inria_' + experiment + '_25_05_windows.csv'
filename_gfrc_windows = 'gfrc_' + experiment + '_25_05_windows.csv'
filename_vedai_windows = 'vedai_' + experiment + '_25_05_windows.csv'

In [ ]:
truth_file_inria = "/data/old_home_dir/ChrissyF/INRIA/00INRIA_bboxes_valid_yolo.csv"
truth_file_gfrc = "/data/old_home_dir/ChrissyF/GFRC/yolo_valid_GFRC_bboxes.csv"
truth_file_vedai = "/data/old_home_dir/ChrissyF/VEDAI/Valid/00VEDAI_test_bboxes.csv"

In [ ]:
valid_whole_image_dir_inria = "/data/old_home_dir/ChrissyF/INRIA/Valid/whole_images/"
valid_whole_image_dir_gfrc = "/data/old_home_dir/ChrissyF/GFRC/Valid/whole_images_all/"
valid_whole_image_dir_vedai = "/data/old_home_dir/ChrissyF/VEDAI/Valid/whole_images/"

In [ ]:
file_dir = "/home/cmf21/pytorch_save/output_for_draft/"

inria_metrics = pd.read_csv(file_dir + filename_inria_metrics)
gfrc_metrics = pd.read_csv(file_dir + filename_gfrc_metrics)
vedai_metrics = pd.read_csv(file_dir + filename_vedai_metrics)
inria_detections = pd.read_csv(file_dir + filename_inria_detections)
gfrc_detections = pd.read_csv(file_dir + filename_gfrc_detections)
vedai_detections = pd.read_csv(file_dir + filename_vedai_detections)
inria_windows = pd.read_csv(file_dir + filename_inria_windows)
vedai_windows = pd.read_csv(file_dir + filename_vedai_windows)
gfrc_windows = pd.read_csv(file_dir + filename_gfrc_windows)
vedai_metrics[vedai_metrics.RE > 0.7]

In [ ]:
gfrc_conf_threshold = 0.25
vedai_conf_threshold = 0.40
inria_conf_threshold = 0.08

In [ ]:
def process_truths(truths, dataset):
    if dataset == 'gfrc':
        truths.loc[:, 'filename'] = [strin.replace('/', '_') for strin in truths.file_loc]
        truths = windows_truth(truths)
    else:
        truths['filename'] = truths['file_loc']
        truths['xmn'] = truths['xmin']
        truths['xmx'] = truths['xmax']
        truths['ymn'] = truths['ymin']
        truths['ymx'] = truths['ymax']
    if dataset == 'vedai':
        truths['oc'] = truths['class_']
    return truths

In [ ]:
inria_truth = pd.read_csv(truth_file_inria)
inria_truth = process_truths(inria_truth, 'inria')
gfrc_truth = pd.read_csv(truth_file_gfrc)
gfrc_truth = process_truths(gfrc_truth, 'gfrc')
vedai_truth = pd.read_csv(truth_file_vedai)
vedai_truth = process_truths(vedai_truth, 'vedai')

In [ ]:
def match_at_threshold(windows_whole, truths, conf_threshold, image_dir, dataset, iou_threshold=0.25, nms_threshold=0.05):
    # get image files
    image_files_jpg = list(Path(image_dir).glob("*.jpg"))
    image_files_png = list(Path(image_dir).glob("*.png"))
    image_files = image_files_jpg + image_files_png
    image_files = [img.name for img in image_files]
    # filter windows
    windows_whole_th = windows_whole[windows_whole.conf > conf_threshold]
    results_all_ims = pd.DataFrame(columns=['xmn', 'xmx', 'ymn', 'ymx', 'conf', 'confmat', 'tru_box', 'filename'])
    tpz = []
    fpz = []
    fnz = []
    for fl in image_files:
        truths_im = truths[truths.filename == fl]

        if dataset == 'gfrc':
            fl_png = fl
        else:
            fl_png = fl[:-4] + '.png'

        windows_whole_im = windows_whole_th[windows_whole_th.filename == fl_png]

        # calculate results
        results_per_im = match_yolo_to_truths(windows_whole_im, truths_im, iou_threshold, nms_threshold)
        results_per_im['filename'] = fl
        tpz.append(np.sum(results_per_im.confmat == 'TP'))
        fpz.append(np.sum(results_per_im.confmat == 'FP'))
        fnz.append(np.sum(results_per_im.confmat == 'FN'))

        results_all_ims = pd.concat((results_all_ims, results_per_im), axis=0, sort=False)
    
    return results_all_ims

In [ ]:
def get_image_stats(detections_df):
    unique_images = np.unique(detections_df.filename)
    tpz = []
    fpz = []
    fnz = []
    for fl in unique_images:
        results_per_im = detections_df[detections_df['filename'] == fl]
        tpz.append(np.sum(results_per_im.confmat == 'TP'))
        fpz.append(np.sum(results_per_im.confmat == 'FP'))
        fnz.append(np.sum(results_per_im.confmat == 'FN'))

    TPz = np.reshape(np.array(tpz), (len(tpz), 1))
    FPz = np.reshape(np.array(fpz), (len(fpz), 1))
    FNz = np.reshape(np.array(fnz), (len(fnz), 1))
    UIz = np.reshape(unique_images, (len(unique_images), 1))
    df_out = pd.DataFrame(np.hstack((UIz, TPz, FPz, FNz)), columns=['filename', 'TP', 'FP', 'FN'])
    
    return df_out

In [ ]:
inria_matched = match_at_threshold(inria_windows, inria_truth, inria_conf_threshold, valid_whole_image_dir_inria, 'inria')
vedai_matched = match_at_threshold(vedai_windows, vedai_truth, vedai_conf_threshold, valid_whole_image_dir_vedai, 'vedai')
gfrc_matched = match_at_threshold(gfrc_windows, gfrc_truth, gfrc_conf_threshold, valid_whole_image_dir_gfrc, 'gfrc')
inria_im_metrics = get_image_stats(inria_matched)
gfrc_im_metrics = get_image_stats(gfrc_matched)
vedai_im_metrics = get_image_stats(vedai_matched)
vedai_matched[vedai_matched.confmat == 'FP']

In [ ]:
inria_tp_best = inria_im_metrics.iloc[np.argmax(np.array(inria_im_metrics.TP)), :]
inria_fp_worst = inria_im_metrics.iloc[np.argmax(np.array(inria_im_metrics.FP)), :]
inria_fn_worst = inria_im_metrics.iloc[np.argmax(np.array(inria_im_metrics.FN)), :]
vedai_tp_best = vedai_im_metrics.iloc[np.argmax(np.array(vedai_im_metrics.TP)), :]
vedai_fp_worst = vedai_im_metrics.iloc[np.argmax(np.array(vedai_im_metrics.FP)), :]
vedai_fn_worst = vedai_im_metrics.iloc[np.argmax(np.array(vedai_im_metrics.FN)), :]
gfrc_tp_best = gfrc_im_metrics.iloc[np.argmax(np.array(gfrc_im_metrics.TP)), :]
gfrc_fp_worst = gfrc_im_metrics.iloc[np.argmax(np.array(gfrc_im_metrics.FP)), :]
gfrc_fn_worst = gfrc_im_metrics.iloc[np.argmax(np.array(gfrc_im_metrics.FN)), :]

In [ ]:
inria_image_outdir = "/home/cmf21/pytorch_save/output_for_draft/" + 'inria_' + experiment + '_25_05/'
vedai_image_outdir = "/home/cmf21/pytorch_save/output_for_draft/" + 'vedai_' + experiment + '_25_05/'
gfrc_image_outdir = "/home/cmf21/pytorch_save/output_for_draft/" + 'gfrc_' + experiment + '_25_05/'

In [ ]:
from pathlib import Path
from PIL import Image
import cv2
from window.utils.drawing import draw_results_on_image

def draw_res(results_all_ims, valid_whole_image_dir, image_out_dir, dataset):
    images_out = []
    fnz_out = []
    fpz_out = []

    image_files_jpg = list(Path(valid_whole_image_dir).glob("*.jpg"))
    image_files_png = list(Path(valid_whole_image_dir).glob("*.png"))
    image_files = image_files_jpg + image_files_png
    image_files = [img.name for img in image_files]

    for fl in image_files:
        # Per image
        whole_im = cv2.imread(valid_whole_image_dir + fl)
        whole_im = cv2.cvtColor(whole_im, cv2.COLOR_BGR2RGB)

        if dataset == 'gfrc':
            fl_png = fl
        else:
            fl_png = fl[:-4] + '.png'

        # calculate results
        results_per_im = results_all_ims[results_all_ims.filename == fl]

        # create list of all false negatives
        for rw in range(results_per_im.shape[0]):
            row = results_per_im.iloc[rw, :]
            if row.confmat == 'FN':
                xmn = max(0, row.xmn - 50)
                ymn = max(0, row.ymn - 50)
                xmx = min(whole_im.shape[1], row.xmx + 50)
                ymx = min(whole_im.shape[0], row.ymx + 50)
                fn_window = whole_im[row.ymn:row.ymx, row.xmn:row.xmx]
                fn_window = cv2.resize(fn_window, (fn_window.shape[1]*2, fn_window.shape[0]*2))
                fnz_out.append(fn_window)
            if row.confmat == 'FP':
                xmn = max(0, row.xmn - 50)
                ymn = max(0, row.ymn - 50)
                xmx = min(whole_im.shape[1], row.xmx + 50)
                ymx = min(whole_im.shape[0], row.ymx + 50)
                fp_window = whole_im[row.ymn:row.ymx, row.xmn:row.xmx]
                f_window = cv2.resize(fp_window, (fp_window.shape[1]*2, fp_window.shape[0]*2))
                fpz_out.append(fp_window)

        # draw results on image
        image_out = draw_results_on_image(whole_im, results_per_im)
        image_out = cv2.resize(image_out, (1840, 1228))
        images_out.append(image_out)
        image_out = cv2.cvtColor(image_out, cv2.COLOR_BGR2RGB)
        cv2.imwrite(str(image_out_dir + fl), image_out)
        
    return fnz_out, fpz_out

In [ ]:
valid_whole_image_dir_inria = "/data/old_home_dir/ChrissyF/INRIA/Valid/whole_images/"
valid_whole_image_dir_gfrc = "/data/old_home_dir/ChrissyF/GFRC/Valid/whole_images_all/"
valid_whole_image_dir_vedai = "/data/old_home_dir/ChrissyF/VEDAI/Valid/whole_images/"

inria_fnz, inria_fpz = draw_res(inria_matched, valid_whole_image_dir_inria, inria_image_outdir, 'inria')
vedai_fnz, vedai_fpz = draw_res(vedai_matched, valid_whole_image_dir_vedai, vedai_image_outdir, 'vedai')
gfrc_fnz, gfrc_fpz = draw_res(gfrc_matched, valid_whole_image_dir_gfrc, gfrc_image_outdir, 'gfrc')

In [ ]:
inria_im_metrics[inria_im_metrics.TP > 4]

In [ ]:
from PIL import Image
# Image.open(inria_image_outdir + 'person_and_bike_181.png') # FP
# Image.open(inria_image_outdir + 'crop001521.png') # FN
Image.open(inria_image_outdir + 'person_236.png') # TP

# Image.open(vedai_image_outdir + '00000127_co.png')
# Image.open(vedai_image_outdir + '00000396_co.png')
# Image.open(vedai_image_outdir + '00000181_co.png')

In [ ]:
import random
def create_mosaic(list_in, mosaic_tuple, image_size, grey=False):
    rowz = mosaic_tuple[0]
    colz = mosaic_tuple[1]
    img_rw = image_size[0]
    img_cl = image_size[1]
    channels = 3
    if grey:
        channels = 1
    combined_im = np.zeros((img_rw * rowz, img_cl * colz, channels), dtype=np.uint8)
    sample_list = random.sample(list_in, rowz*colz)
    for idx, im in enumerate(sample_list):
        im_reshape = cv2.resize(im, (img_cl, img_rw))
        col = idx % colz
        row = idx // colz
        x1 = col * img_cl
        x2 = (col + 1) * img_cl
        y1 = row * img_rw
        y2 = (row + 1) * img_rw
        combined_im[y1:y2, x1:x2, :] = im_reshape
        
    return combined_im

In [ ]:
fnz_combined = create_mosaic(vedai_fnz, (4, 7), (60*2, 60*2))
Image.fromarray(fnz_combined)

In [ ]:
fpz_combined = create_mosaic(vedai_fpz, (4, 7), (60*2, 60*2))
Image.fromarray(fpz_combined)

In [ ]:
fnz_combined = create_mosaic(inria_fnz, (2, 7), (128*2, 64*2))
Image.fromarray(fnz_combined)

In [ ]:
fpz_combined = create_mosaic(inria_fpz, (2, 7), (128*2, 64*2))
Image.fromarray(fpz_combined)

In [ ]:
inria_windows

In [ ]:
def nms_per_im(boxes_in, thresh, method='first'):
    
    boxes_in = boxes_in.sort_values(by='conf', ascending=False)

    xmins = boxes_in.xmn
    xmaxs = boxes_in.xmx
    ymins = boxes_in.ymn
    ymaxs = boxes_in.ymx
    confs = boxes_in.conf

    boxes_ot = pd.DataFrame(columns=['xmn', 'xmx', 'ymn', 'ymx', 'conf'])

    xmins = np.array(xmins.tolist())
    xmaxs = np.array(xmaxs.tolist())
    ymins = np.array(ymins.tolist())
    ymaxs = np.array(ymaxs.tolist())
    confs = np.array(confs.tolist())

    while len(xmins) > 0:

        xmn = xmins[0]
        xmns = np.array(xmins[1:])
        xmx = xmaxs[0]
        xmxs = np.array(xmaxs[1:])
        ymn = ymins[0]
        ymns = np.array(ymins[1:])
        ymx = ymaxs[0]
        ymxs = np.array(ymaxs[1:])
        cnf = confs[0]
        cnfs = np.array(confs[1:])

        ol_wid = np.minimum(xmx, xmxs) - np.maximum(xmn, xmns)
        ol_hei = np.minimum(ymx, ymxs) - np.maximum(ymn, ymns)

        ol_x = np.maximum(0, ol_wid)
        ol_y = np.maximum(0, ol_hei)

        distx = np.subtract(xmxs, xmns)
        disty = np.subtract(ymxs, ymns)
        bxx = xmx - xmn
        bxy = ymx - ymn

        ol_area = np.multiply(ol_x, ol_y)
        bx_area = bxx * bxy
        bxs_area = np.multiply(distx, disty)

        ious = np.divide(ol_area, np.subtract(np.add(bxs_area, bx_area), ol_area))
        mask_bxs = np.greater(ious, thresh)

        if np.sum(mask_bxs) > 0:
            box_ot = pd.DataFrame(index=range(1), columns=['xmn', 'xmx', 'ymn', 'ymx', 'conf'])

            xmns = xmns[mask_bxs]
            xmxs = xmxs[mask_bxs]
            ymns = ymns[mask_bxs]
            ymxs = ymxs[mask_bxs]
            cnfs = cnfs[mask_bxs]

            if method == 'mean':
                box_ot.loc[0, 'xmn'] = np.array(np.mean(xmns), dtype=int)
                box_ot.loc[0, 'ymn'] = np.array(np.mean(ymns), dtype=int)
                box_ot.loc[0, 'xmx'] = np.array(np.mean(xmxs), dtype=int)
                box_ot.loc[0, 'ymx'] = np.array(np.mean(ymxs), dtype=int)
                box_ot.loc[0, 'conf'] = np.mean(cnfs)
            elif method == 'first':
                box_ot.loc[0, 'xmn'] = xmns[0]
                box_ot.loc[0, 'ymn'] = ymns[0]
                box_ot.loc[0, 'xmx'] = xmxs[0]
                box_ot.loc[0, 'ymx'] = ymxs[0]
                box_ot.loc[0, 'conf'] = np.max(cnfs)
            else:
                box_ot.loc[0, 'xmn'] = np.min(xmns)
                box_ot.loc[0, 'ymn'] = np.min(ymns)
                box_ot.loc[0, 'xmx'] = np.max(xmxs)
                box_ot.loc[0, 'ymx'] = np.max(ymxs)
                box_ot.loc[0, 'conf'] = np.mean(cnfs)

            mask_out = np.repeat(False, len(xmins))
            mask_out[0] = True
            mask_out[1:] = mask_bxs
            mask_out = np.logical_not(mask_out)

            xmins = xmins[mask_out]
            xmaxs = xmaxs[mask_out]
            ymins = ymins[mask_out]
            ymaxs = ymaxs[mask_out]
            confs = confs[mask_out]
            
        else:
            box_ot = pd.DataFrame(index=range(1), columns=['xmn', 'xmx', 'ymn', 'ymx', 'conf'])

            box_ot.loc[0, 'xmn'] = xmn
            box_ot.loc[0, 'ymn'] = ymn
            box_ot.loc[0, 'xmx'] = xmx
            box_ot.loc[0, 'ymx'] = ymx
            box_ot.loc[0, 'conf'] = cnf

            mask_out = np.repeat(False, len(xmins))
            mask_out[0] = True
            mask_out = np.logical_not(mask_out)
            
            xmins = xmins[mask_out]
            xmaxs = xmaxs[mask_out]
            ymins = ymins[mask_out]
            ymaxs = ymaxs[mask_out]
            confs = confs[mask_out]
            
        #box_ot = box_ot.reset_index(drop=True)
        boxes_ot = pd.concat((boxes_ot, box_ot), axis=0, sort=False)

    boxes_ot.loc[:, 'filename'] = boxes_in.filename.iloc[0]

    return boxes_ot

In [ ]:
def nms_for_yolo(windows_df, nms_thresh):
    images = np.unique(windows_df.filename)
    windows_all_ims = pd.DataFrame(columns=['xmn', 'xmx', 'ymn', 'ymx', 'conf', 'filename'])
    for im in images:
        windows_im = windows_df[windows_df.filename == im]
        windows_im = nms_per_im(windows_im, nms_thresh)
        windows_all_ims.append(windows_im)
        windows_all_ims = pd.concat((windows_all_ims, windows_im), axis=0, ignore_index=True, sort=False)
    return windows_all_ims
    

In [ ]:
conf_threshold = 0.25
nms_threshold = 0.05

inria_windows_th = inria_windows[inria_windows.conf >= conf_threshold]
inria_windows_nms = nms_for_yolo(inria_windows_th, nms_threshold)

inria_windows_nms

In [ ]:
inria_im = inria_windows[inria_windows.filename == 'D2004-08-21_10h49m39s.jpg']
inria_detections_im = nms_per_im(inria_im, 0.05)
inria_detections_im